In [1]:
import csv
import json
import feather # pip install feather-format
import numpy as np
import pandas as pd
import os
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import geopandas as gpd
import ast
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,LogisticRegression
%matplotlib inline


In [2]:
## Some utility functions
def percentToInt(percentString):
    return int(percentString.rstrip("%"))
def trueFalseToBinary(tfstring):
    return 1 if tfstring=='t' else '0'
def stringListToList(stringList):
    return ast.literal_eval(stringList)
def stringListToCount(stringList):
    return len(ast.literal_eval(stringList))

In [4]:
reviews_data_path = "feather/reviews.feather"
listings_data_path = "feather/listings.feather"
calendar_data_path = "feather/calendar.feather"

reviews_data = pd.read_feather(reviews_data_path)
listings_data = pd.read_feather(listings_data_path)
calendar_data = pd.read_feather(calendar_data_path)

In [6]:
features_none_null = ["id","host_id","host_since","host_response_rate","host_acceptance_rate",
                        "host_response_time",
                        "host_is_superhost", "host_verifications",
                        # "host_has_profile_pic", "host_identity_verified",
                        # "neighbourhood_cleansed",
                        "latitude", "longitude",
                        "room_type", "accommodates", "amenities", "price", "minimum_nights", 
                        "availability_30",	"availability_60",	"availability_90","availability_365", 
                        "number_of_reviews", "review_scores_rating", 
                        "review_scores_accuracy",	"review_scores_cleanliness",	"review_scores_checkin",	"review_scores_communication",
                        "review_scores_location",	"review_scores_value",
                        "instant_bookable", 
                        ]
reviews_data_path = "feather/reviews.feather"
listings_data_path = "feather/listings.feather"
calendar_data_path = "feather/calendar.feather"
reviews_data = pd.read_feather(reviews_data_path)
listings_data = pd.read_feather(listings_data_path)
calendar_data = pd.read_feather(calendar_data_path)

# reviews_data_path = "data/reviews.csv"
# listings_data_path = "data/listings.csv"
# calendar_data_path = "data/calendar.csv"
# reviews_data = pd.read_csv(reviews_data_path)
# listings_data = pd.read_csv(listings_data_path)
# calendar_data = pd.read_csv(calendar_data_path)

reviews_data = reviews_data.dropna()

df_none_null = listings_data.copy(deep = True)[features_none_null]

print("before droppign rows with null columns: ", df_none_null.shape)
for col in features_none_null:
    df_none_null = df_none_null[df_none_null[col].notna()]
print("after dropping rows with null columns: ", df_none_null.shape)


## most null columns - host_response_rate/accpetance rate, review_scores

reference_date = datetime.datetime(2021,7,12)
# adding custom: host_number_of_years
host_number_of_years = [(reference_date - datetime.datetime.strptime(i, '%Y-%m-%d')).days/365
                        for i in df_none_null['host_since']]
df_none_null["host_number_of_years"] = host_number_of_years
df_none_null['price'] = df_none_null['price'].map(lambda x:float(x[1:].replace(',', ''))).astype('int32')


# host reposnse rate; remove % sign, convert to integer
df_none_null['host_response_rate'] = df_none_null['host_response_rate'].map(percentToInt).astype('int32')
#  host_acceptance_rate:
df_none_null['host_acceptance_rate'] = df_none_null['host_acceptance_rate'].map(percentToInt).astype('int32')

# TODO host_response_time; arbitrary or one-hot
# df_none_null['host_response_time']= df_none_null['host_response_time'].map({
#     'within an hour':1, 'within a day':3, 'within a few hours':2,
#        'a few days or more':4
# })
host_response_time_dummies = pd.get_dummies(df_none_null['host_response_time'],prefix='host_response_time')
df_none_null = pd.concat([df_none_null,host_response_time_dummies], axis = 1)

# host_is_superhost: t/f to binary
df_none_null['host_is_superhost'] = df_none_null['host_is_superhost'].map(trueFalseToBinary).astype('int32')
# "host_verifications",
df_none_null['host_verifications'] = df_none_null['host_verifications'].map(stringListToCount).astype('int32')
# # "host_has_profile_pic" tf to binary
# df_none_null['host_has_profile_pic'] = df_none_null['host_has_profile_pic'].map(trueFalseToBinary).astype('int32')
# # "host_identity_verified",tf to binary
# df_none_null['host_identity_verified'] = df_none_null['host_identity_verified'].map(trueFalseToBinary).astype('int32')

# TODO "neighbourhood_cleansed" or leave out - lat long enough

# "latitude" pass
# "longitude", pass

# "room_type" ['Entire home/apt'4 'Private room'2 'Shared room'1 'Hotel room'3]
df_none_null['room_type'] = df_none_null['room_type'].map(
                            {'Entire home/apt':4,
                                'Private room':2,
                                 'Shared room':1,
                                  'Hotel room':3})

# "accommodates", pass
#  "amenities", 
df_none_null['amenities'] = df_none_null['amenities'].map(stringListToCount).astype('int32')
# "price", pass

# TODO "minimum_nights", to categorical? 3 categories [short medium long?]
# df_none_null['minimum_nights'] = df_none_null['minimum_nights']

# "availability_30",
df_none_null['availability_30'] = df_none_null['availability_30']/30.0
# "availability_60",
df_none_null['availability_60'] = df_none_null['availability_60']/60.0
# "availability_90",
df_none_null['availability_90'] = df_none_null['availability_90']/90.0
# "availability_365", 
df_none_null['availability_365'] = df_none_null['availability_365']/365.0

# "number_of_reviews" pass,

# all pass
# "review_scores_rating", 
# "review_scores_accuracy",
# "review_scores_cleanliness",
# 	"review_scores_checkin",	
# "review_scores_communication",
# "review_scores_location",
# 	"review_scores_value",

# "instant_bookable", 
df_none_null['instant_bookable'] = df_none_null['instant_bookable'].map(trueFalseToBinary).astype('int32')

df_none_null = df_none_null.drop(['host_response_time', 'host_since'], axis=1)


before droppign rows with null columns:  (3043, 28)
after dropping rows with null columns:  (1678, 28)


# Engineer more VARIABLEs here

In [37]:
df_none_null = df_none_null[(np.abs(stats.zscore(df_none_null['price'])) < 2)]

In [38]:
print(df_none_null['price'].describe())

count    1528.000000
mean      139.899869
std        77.141480
min         0.000000
25%        80.000000
50%       125.000000
75%       186.000000
max       355.000000
Name: price, dtype: float64


## Standarize features
### TODO: make certain binary features non-standardized

In [40]:
df = df_none_null.drop(['price', 'id','host_id'], axis=1)
df_normalized=(df - df.mean()) / df.std()
df_label = df_none_null['price']
X_train, X_test, y_train, y_test = train_test_split(df, df_label, test_size=0.1, random_state=101)



In [63]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
knn_model = KNeighborsClassifier(n_neighbors=2)
knn_model.fit(X_train, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt
train_preds = knn_model.predict(X_train)
mse = mean_squared_error(y_train, train_preds)
rmse = sqrt(mse)
rmse

62.34428165072924

In [64]:
train_preds

array([ 90,  87,  72, ...,  50, 115, 195], dtype=int32)

In [24]:
test_preds = knn_model.predict(X_test)
mse = mean_squared_error(y_test, test_preds)
rmse = sqrt(mse)
rmse

240.00939720634796

# Neural Network


In [65]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

# https://www.tensorflow.org/tutorials/keras/regression
# sns.pairplot(train_dataset[['MPG', 'Cylinders', 'Displacement', 'Weight']], diag_kind='kde')

In [93]:
# X_train, X_test, y_train, y_test 
df = df_none_null.drop(['id','host_id'], axis=1)
df_normalized=(df - df.mean()) / df.std()
df_label = df_normalized['price']
X_train, X_test, y_train, y_test = train_test_split(df_normalized, df_label, test_size=0.1, random_state=101)

nnn = 200
model = keras.Sequential(
    [
        keras.Input(shape=X_train.shape),
        layers.Dense(nnn,activation="relu"),
        layers.Dense(nnn,activation="relu"),
        layers.Dense(nnn,activation="relu"),
        layers.Dense(nnn,activation="relu"),
        layers.Dense(nnn,activation="relu"),
        layers.Dense(nnn,activation="relu"),
        layers.Dense(1),
    ]
)
#@title 4. dense model compile
batch_size = 300
epochs = 200
op = keras.optimizers.SGD(learning_rate = 0.1)
# op = keras.optimizers.Adam(learning_rate = 0.005)
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=op, metrics=tf.keras.metrics.MeanSquaredError())

model.fit(X_train, y_train.to_numpy(), batch_size=batch_size, epochs=epochs, validation_split=0.1)



Epoch 1/200
5/5 [==============================] - 1s 54ms/step - loss: 0.9509 - mean_squared_error: 0.9509 - val_loss: 0.8130 - val_mean_squared_error: 0.8130
Epoch 2/200
5/5 [==============================] - 0s 7ms/step - loss: 0.9007 - mean_squared_error: 0.9007 - val_loss: 0.7648 - val_mean_squared_error: 0.7648
Epoch 3/200
5/5 [==============================] - 0s 7ms/step - loss: 0.4811 - mean_squared_error: 0.4811 - val_loss: 0.2455 - val_mean_squared_error: 0.2455
Epoch 4/200
5/5 [==============================] - 0s 7ms/step - loss: 0.1882 - mean_squared_error: 0.1882 - val_loss: 2.0532 - val_mean_squared_error: 2.0532
Epoch 5/200
5/5 [==============================] - 0s 7ms/step - loss: 1.0741 - mean_squared_error: 1.0741 - val_loss: 0.6949 - val_mean_squared_error: 0.6949
Epoch 6/200
5/5 [==============================] - 0s 7ms/step - loss: 0.5454 - mean_squared_error: 0.5454 - val_loss: 0.4451 - val_mean_squared_error: 0.4451
Epoch 7/200
5/5 [============================

In [100]:
model.evaluate(X_test, y_test, verbose=0)


[0.01062995009124279, 0.01062995009124279]

In [119]:
result = model.predict(X_test)
converted_rest = model.predict(X_test)*df_none_null['price'].std()+df_none_null['price'].mean()

difff= []
for i, j in enumerate(y_test*df_none_null['price'].std()+df_none_null['price'].mean()):
    difff.append(abs(converted_rest[i][0] - j))

In [114]:
np.array(difff).sum()

12.13723529479881

In [120]:
difff

[7.3625640869140625,
 1.2683563232421875,
 6.3058624267578125,
 2.4920654296875,
 1.4418487548828125,
 18.15087890625,
 2.8731765747070312,
 15.101531982421875,
 16.903060913085938,
 6.6609344482421875,
 1.76385498046875,
 7.4946746826171875,
 5.43426513671875,
 12.888916015625,
 6.615142822265625,
 3.1170196533203125,
 4.07904052734375,
 3.25732421875,
 13.140625,
 0.8580780029296875,
 3.235870361328125,
 5.016517639160156,
 3.78155517578125,
 8.6981201171875,
 1.5047378540039062,
 2.7255706787109375,
 6.236663818359375,
 3.761016845703125,
 0.3214569091796875,
 7.816047668457031,
 10.006484985351562,
 2.1675262451171875,
 22.01904296875,
 9.184761047363281,
 1.444671630859375,
 7.2379913330078125,
 5.3543548583984375,
 12.484451293945312,
 6.9899444580078125,
 2.1557693481445312,
 5.698699951171875,
 2.4878311157226562,
 9.536270141601562,
 5.96136474609375,
 3.8722686767578125,
 7.655609130859375,
 4.519500732421875,
 8.255233764648438,
 4.111274719238267,
 6.8006515502929545,
 7.86

In [117]:
converted_rest = model.predict(X_test)*df_none_null['price'].std()+df_none_null['price'].mean()

In [118]:
converted_rest

array([[130.36256 ],
       [220.73164 ],
       [ 85.30586 ],
       [ 51.507935],
       [212.55815 ],
       [121.15088 ],
       [ 51.873177],
       [184.10153 ],
       [121.90306 ],
       [142.33907 ],
       [119.763855],
       [164.50533 ],
       [202.43427 ],
       [ 88.888916],
       [ 56.615143],
       [166.88298 ],
       [295.92096 ],
       [ 58.257324],
       [115.140625],
       [124.14192 ],
       [148.23587 ],
       [ 37.016518],
       [285.21844 ],
       [223.69812 ],
       [ 46.504738],
       [152.72557 ],
       [208.23666 ],
       [219.76102 ],
       [109.67854 ],
       [ 37.816048],
       [ 63.006485],
       [ 58.167526],
       [254.01904 ],
       [ 96.18476 ],
       [ 60.55533 ],
       [ 38.23799 ],
       [147.35435 ],
       [161.48445 ],
       [ 81.989944],
       [ 74.84423 ],
       [324.6987  ],
       [ 97.48783 ],
       [114.53627 ],
       [ 74.961365],
       [148.87227 ],
       [116.65561 ],
       [112.4805  ],
       [ 69.7